In [23]:
import pandas as pd
import os
import re

# ==============================================================================
# HÀM CHUẨN HÓA VÀ HÀM TRỢ GIÚP (Giữ nguyên)
# ==============================================================================

def normalize_text(text):
    if pd.isna(text):
        return ""
    cleaned_text = re.sub(r'\s*\(.*?\)\s*', ' ', str(text))
    final_text = re.sub(r'\s+', ' ', cleaned_text).strip().lower()
    return final_text

def add_normalized_keys(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['key_title'] = df_copy['title'].apply(normalize_text) if 'title' in df_copy.columns else ''
    df_copy['key_acronym'] = df_copy['acronym'].apply(normalize_text) if 'acronym' in df_copy.columns else ''
    return df_copy

# ==============================================================================
# HÀM ĐẾM SỐ LƯỢNG DUY NHẤT (Giữ nguyên)
# ==============================================================================

def calculate_and_print_total_unique_count(all_files: list):
    total_unique_conferences = set()
    print("\n" + "="*80)
    print(" BƯỚC 0: TÍNH TOÁN TỔNG SỐ CONFERENCE DUY NHẤT TRÊN TẤT CẢ CÁC FILE")
    print("="*80)
    for file_path in all_files:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=['', ' '], dtype=str, low_memory=False)
            df_normalized = add_normalized_keys(df)
            if not df_normalized.empty:
                unique_keys_in_file = set(zip(df_normalized['key_title'], df_normalized['key_acronym']))
                total_unique_conferences.update(unique_keys_in_file)
                print(f"Đã xử lý file để đếm: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"Lỗi đọc file {os.path.basename(file_path)} trong quá trình đếm: {e}")
    print("\n" + "-"*40)
    print(f">>> TỔNG SỐ CONFERENCE DUY NHẤT (dựa trên title và acronym đã chuẩn hóa) là: {len(total_unique_conferences)}")
    print("-"*40)

# ==============================================================================
# HÀM XỬ LÝ CHÍNH THEO LOGIC MỚI
# ==============================================================================
def process_by_any_correct_version(
    input_files: list,
    aggregated_correct_path: str,
    aggregated_recrawl_path: str,
    recrawled_files: list = None
):
    """
    Phân loại conference: nếu có bất kỳ phiên bản nào không có 'note',
    nó sẽ được coi là 'correct' và không nằm trong danh sách 'recrawl'.
    Luôn ưu tiên lấy phiên bản mới nhất.
    """
    print("\n" + "="*80)
    print(" BẮT ĐẦU KỊCH BẢN: ƯU TIÊN BẤT KỲ PHIÊN BẢN ĐÚNG NÀO")
    print("="*80 + "\n")

    # --- BƯỚC 1: ĐỌC VÀ GỘP TẤT CẢ CÁC FILE (ƯU TIÊN FILE ĐÃ RECRAWL) ---
    print("--- BƯỚC 1: Đọc và gộp tất cả các file ---")
    all_dfs = []
    all_files_in_order = (recrawled_files or []) + (input_files or [])
    
    for file_path in all_files_in_order:
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=[''], dtype=str).fillna('')
            df = add_normalized_keys(df)
            df = df[df['key_title'] != ''].copy()
            if not df.empty:
                all_dfs.append(df)
        except Exception as e:
            print(f"Lỗi xử lý file {file_path}: {e}")
            continue
            
    if not all_dfs:
        print("Không có dữ liệu để xử lý. Dừng chương trình.")
        return

    master_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Đã gộp thành công. Tổng số dòng để xử lý: {len(master_df)}")

    # --- BƯỚC 2: XÁC ĐỊNH DANH SÁCH CÁC CONFERENCE "ĐÚNG" ---
    print("\n--- BƯỚC 2: Xác định danh sách các conference có ít nhất một phiên bản đúng ---")
    key_cols = ['key_title', 'key_acronym']
    
    # Điều kiện một dòng được coi là "đúng"
    is_correct_condition = ~master_df['note'].notna() | (master_df['note'].astype(str).str.strip() == '')
    
    # Lấy tất cả các dòng đúng
    all_correct_versions = master_df[is_correct_condition]
    
    # Lấy danh sách các khóa duy nhất của các conference đúng
    correct_keys_df = all_correct_versions.drop_duplicates(subset=key_cols, keep='first')
    correct_keys_set = set(zip(correct_keys_df['key_title'], correct_keys_df['key_acronym']))
    print(f"Tìm thấy {len(correct_keys_set)} conference có ít nhất một phiên bản đúng.")

    # --- BƯỚC 3: TẠO FILE CORRECT CUỐI CÙNG ---
    print("\n--- BƯỚC 3: Tạo file correct cuối cùng ---")
    # Từ tất cả các phiên bản đúng, giữ lại phiên bản mới nhất (xuất hiện đầu tiên) cho mỗi conference
    final_correct_df = all_correct_versions.drop_duplicates(subset=key_cols, keep='first')
    
    # --- BƯỚC 4: TẠO FILE RECRAWL CUỐI CÙNG ---
    print("\n--- BƯỚC 4: Tạo file recrawl cuối cùng ---")
    # Lấy tất cả các dòng có note
    all_recrawl_versions = master_df[~is_correct_condition].copy()
    
    if not all_recrawl_versions.empty:
        # Loại bỏ những conference đã có trong danh sách đúng
        mask_keep = ~all_recrawl_versions.apply(
            lambda row: (row['key_title'], row['key_acronym']) in correct_keys_set,
            axis=1
        )
        recrawl_to_keep = all_recrawl_versions[mask_keep]
        
        # Từ những dòng còn lại, giữ lại phiên bản mới nhất
        final_recrawl_df = recrawl_to_keep.drop_duplicates(subset=key_cols, keep='first')
    else:
        final_recrawl_df = pd.DataFrame(columns=master_df.columns)

    # --- BƯỚC 5: LƯU FILE ---
    print("\n--- BƯỚC 5: Hoàn tất và lưu file tổng hợp cuối cùng ---")
    
    # Dọn dẹp cột khóa trước khi lưu
    final_correct_df = final_correct_df.drop(columns=key_cols, errors='ignore')
    final_recrawl_df = final_recrawl_df.drop(columns=key_cols, errors='ignore')

    # Lưu file CORRECT
    final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
    print(f"Đã lưu tổng cộng {len(final_correct_df)} dòng vào file correct '{aggregated_correct_path}'.")

    # Lưu file RECRAWL
    final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
    print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng vào file recrawl '{aggregated_recrawl_path}'.")


# --- Cách sử dụng ---
if __name__ == "__main__":
    input_csv_files_for_note_check = [


        './src/conference/evaluate/batch2.csv',
        './src/conference/evaluate/batch3.csv',
        './src/conference/evaluate/batch8.csv',
        './src/conference/evaluate/batch12.csv',
        './src/conference/evaluate/batch13.csv',
        './src/conference/evaluate/batch16.csv',
        './src/conference/evaluate/batch19.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_139_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_lan_3.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_1_50.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_51_end.csv',



        
        './src/conference/evaluate/batch4.csv',
        './src/conference/evaluate/batch5.csv',
        './src/conference/evaluate/batch9.csv',
        './src/conference/evaluate/batch14.csv',
        './src/conference/evaluate/batch15.csv',
        './src/conference/evaluate/batch18.csv',
        './src/conference/evaluate/batch20.csv',
        './src/conference/evaluate/recrawl_all_thang_1_50_lan_1_check.csv',
        './src/conference/evaluate/recrawl_all_thang_51_58_lan_1_check.csv',
        
        './src/conference/evaluate/evaluate_recrawl_all_tri_1_50_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_51_100_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_101_120_lan_5.csv',
        
        './src/conference/evaluate/evaluate_crawl_con_lai_lan_2.csv',
        './src/conference/evaluate/evaluate_crawl_con_lai.csv',
        './src/conference/evaluate/evaluate_crawl_con_lai_lan_3.csv',
        './src/conference/evaluate/evaluate_crawl_missing_info_in_full.csv',


    ]
    recrawled_results_files = [

        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_139_lan_2.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_lan_3.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50_lan_4.csv',


        './src/conference/evaluate/evaluate_crawl_not_crawl_1_50.csv',
        './src/conference/evaluate/evaluate_crawl_not_crawl_51_end.csv',


        
        './src/conference/evaluate/recrawl_batch_4_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_5_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_9_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_14_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_15_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_18_check_lan_1.csv',
        './src/conference/evaluate/recrawl_batch_20_check_lan_1.csv',
        './src/conference/evaluate/recrawl_all_tri_1_50.csv',
        './src/conference/evaluate/recrawl_all_tri_51_100.csv',
        './src/conference/evaluate/recrawl_all_tri_101_150.csv',
        './src/conference/evaluate/recrawl_all_tri_1_32_lan_2.csv',
        './src/conference/evaluate/recrawl_all_tri_lan_3.csv',
        './src/conference/evaluate/recrawl_all_tri_lan_4.csv',

        './src/conference/evaluate/evaluate_recrawl_all_tri_1_50_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_51_100_lan_5.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tri_101_120_lan_5.csv',



       

    ]
    aggregated_correct_file_note = './src/conference/evaluate/tri_tung_all_correct_final.csv'
    aggregated_recrawl_file_note = './src/conference/evaluate/tri_tung_all_recrawl_final.csv'

    all_files_for_counting = recrawled_results_files + input_csv_files_for_note_check
    calculate_and_print_total_unique_count(all_files_for_counting)

    process_by_any_correct_version(
        input_files=input_csv_files_for_note_check,
        aggregated_correct_path=aggregated_correct_file_note,
        aggregated_recrawl_path=aggregated_recrawl_file_note,
        recrawled_files=recrawled_results_files
    )


 BƯỚC 0: TÍNH TOÁN TỔNG SỐ CONFERENCE DUY NHẤT TRÊN TẤT CẢ CÁC FILE
Đã xử lý file để đếm: evaluate_recrawl_batch_2_3_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_8_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_12_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_13_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_16_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_batch_19_lan_1.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_1_50.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_51_100.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_101_150.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_151_159.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_1_50_lan_2.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_51_100_lan_2.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_101_139_lan_2.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_lan_3.csv
Đã xử lý file để đếm: evaluate_recrawl_all_tung_1_50_lan_4.csv
Đã xử lý file để đếm: evaluate_r

In [45]:
import pandas as pd
import os
import json

def load_non_link_acronyms(filepath):
    """
    Đọc file non_link.txt và trả về một set chứa các acronym.

    Args:
        filepath (str): Đường dẫn đến file non_link.txt.

    Returns:
        set: Một set chứa các acronym từ file.
    """
    acronyms = set()
    if not os.path.exists(filepath):
        print(f"Cảnh báo: File non_link.txt không tồn tại tại '{filepath}'. Không áp dụng lọc non-link.")
        return acronyms
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                acronyms.add(line.strip())
        print(f"Đã tải {len(acronyms)} acronym từ '{filepath}'.")
    except Exception as e:
        print(f"Lỗi khi đọc file non_link.txt '{filepath}': {e}. Không áp dụng lọc non-link.")
    return acronyms

def merge_csv_common_columns_ordered(file_paths, output_filename, non_link_acronyms_filepath=None):
    """
    Đọc nhiều file CSV, tìm các cột chung (phân biệt hoa thường),
    sau đó hợp nhất các file chỉ với các cột chung đó.
    Thứ tự các cột trong file kết quả sẽ theo thứ tự của các cột chung
    trong file CSV cuối cùng trong danh sách input.
    Sau đó, loại bỏ các dòng có acronym nằm trong danh sách non-link.

    Args:
        file_paths (list): Danh sách các đường dẫn đầy đủ đến các file CSV.
        output_filename (str): Đường dẫn và tên file CSV đầu ra.
        non_link_acronyms_filepath (str, optional): Đường dẫn đến file non_link.txt.
                                                    Nếu được cung cấp, các dòng có acronym trong file này sẽ bị loại bỏ.
    
    Returns:
        bool: True nếu merge thành công, False nếu có lỗi.
    """
    if not file_paths:
        print("Lỗi: Danh sách đường dẫn file CSV trống. Không có gì để xử lý.")
        return False

    dataframes = []
    # Bước 1: Đọc tất cả các file CSV vào DataFrame
    for fp in file_paths:
        try:
            # Thêm dtype=str để đảm bảo mọi thứ được đọc vào dưới dạng chuỗi,
            # tránh việc pandas tự động chuyển đổi kiểu dữ liệu có thể làm hỏng JSON.
            df = pd.read_csv(fp, dtype=str)
            dataframes.append(df)
            print(f"Đã đọc thành công: {fp} (có {len(df.columns)} cột)")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file tại đường dẫn: {fp}. Bỏ qua file này.")
            continue
        except pd.errors.EmptyDataError:
            print(f"Cảnh báo: File trống hoặc không có header: {fp}. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Lỗi khi đọc file {fp}: {e}. Bỏ qua file này.")
            continue

    if not dataframes:
        print("Không có file CSV nào được đọc thành công. Không thể thực hiện merge.")
        return False

    # Bước 2: Tìm danh sách các cột chung có ở tất cả các file
    common_columns_set = set(dataframes[0].columns)
    for i in range(1, len(dataframes)):
        common_columns_set.intersection_update(set(dataframes[i].columns))

    if not common_columns_set:
        print("Không tìm thấy cột chung nào ở TẤT CẢ các file CSV đã đọc. Không tạo file mới.")
        return False

    # Bước 3: Lấy thứ tự cột từ file cuối cùng trong danh sách
    last_df = dataframes[-1]
    ordered_common_columns = [col for col in last_df.columns if col in common_columns_set]
    
    if len(ordered_common_columns) != len(common_columns_set):
        print("Cảnh báo: Thứ tự cột từ file cuối cùng không chứa tất cả các cột chung. Sẽ sắp xếp theo alphabet.")
        ordered_common_columns = sorted(list(common_columns_set))

    print(f"\nTìm thấy {len(ordered_common_columns)} cột chung ở tất cả các file:")
    print(f"Thứ tự cột trong file output sẽ dựa trên file cuối cùng: {file_paths[-1]}")
    print(f"Thứ tự các cột: {ordered_common_columns}")

    # Bước 4: Tạo danh sách các DataFrame mới, chỉ chứa các cột chung
    filtered_dataframes = [df[ordered_common_columns] for df in dataframes]

    # Bước 5: Hợp nhất tất cả các DataFrame đã lọc lại với nhau
    try:
        merged_df = pd.concat(filtered_dataframes, ignore_index=True)
        print(f"DataFrame đã hợp nhất ban đầu có {merged_df.shape[0]} dòng.")

        # Bước 5.5: Lọc bỏ các dòng có acronym nằm trong non_link.txt
        if non_link_acronyms_filepath and 'acronym' in merged_df.columns:
            non_link_acronyms = load_non_link_acronyms(non_link_acronyms_filepath)
            if non_link_acronyms:
                initial_rows = len(merged_df)
                merged_df = merged_df[~merged_df['acronym'].isin(non_link_acronyms)]
                removed_rows = initial_rows - len(merged_df)
                if removed_rows > 0:
                    print(f"Đã loại bỏ {removed_rows} dòng có acronym trong non_link.txt.")
                else:
                    print("Không có dòng nào được loại bỏ theo non_link.txt.")
            else:
                print("Không có acronym nào để lọc từ non_link.txt hoặc file không hợp lệ.")
        elif 'acronym' not in merged_df.columns and non_link_acronyms_filepath:
            print("Cảnh báo: Cột 'acronym' không tồn tại trong DataFrame đã hợp nhất. Không thể lọc non-link.")

        # Bước 6: Lưu DataFrame đã hợp nhất và lọc vào file CSV mới
        merged_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\nĐã hợp nhất và lọc thành công các file vào '{output_filename}'.")
        print(f"File kết quả có {merged_df.shape[0]} dòng và {merged_df.shape[1]} cột.")
        return True
    except Exception as e:
        print(f"Lỗi khi hợp nhất các DataFrame hoặc ghi file: {e}")
        return False

def validate_json_in_date_columns(csv_filepath):
    """
    Kiểm tra các cột kết thúc bằng "Date" trong một file CSV để xem mỗi ô
    có chứa một chuỗi JSON hợp lệ hay không.

    Args:
        csv_filepath (str): Đường dẫn đến file CSV cần kiểm tra.
    """
    print("\n--- Bắt đầu kiểm tra định dạng JSON trong các cột 'Date' ---")
    try:
        df = pd.read_csv(csv_filepath, dtype=str).fillna('') # Đọc mọi thứ dạng chuỗi, và thay NaN bằng chuỗi rỗng
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{csv_filepath}' để kiểm tra.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file '{csv_filepath}': {e}")
        return

    # Tìm các cột có tên kết thúc bằng "Date"
    date_columns = [col for col in df.columns if col.endswith("Date")]

    if not date_columns:
        print("Không tìm thấy cột nào kết thúc bằng 'Date' để kiểm tra.")
        return

    print(f"Các cột sẽ được kiểm tra: {date_columns}")

    invalid_entries = []
    # Lặp qua từng cột cần kiểm tra
    for col_name in date_columns:
        # Lặp qua từng dòng (index và value) trong cột đó
        for index, value in df[col_name].items():
            # Bỏ qua các ô rỗng hoặc chỉ có khoảng trắng
            if not value or value.isspace():
                continue

            try:
                # Thử phân tích chuỗi thành JSON
                json.loads(value)
            except json.JSONDecodeError:
                # Nếu thất bại, ghi lại thông tin lỗi
                invalid_entries.append({
                    "row_index": index,
                    "column": col_name,
                    "value": value
                })

    # In kết quả
    if not invalid_entries:
        print("\n>>> KIỂM TRA THÀNH CÔNG: Tất cả các giá trị trong các cột 'Date' đều là JSON hợp lệ (hoặc rỗng).")
    else:
        print(f"\n>>> KIỂM TRA THẤT BẠI: Tìm thấy {len(invalid_entries)} giá trị không phải là JSON hợp lệ:")
        print("-" * 50)
        for error in invalid_entries:
            print(f"  - Dòng (chỉ số 0): {error['row_index']}")
            print(f"    Cột              : '{error['column']}'")
            print(f"    Giá trị không hợp lệ: {error['value']}")
            print("-" * 20)

def create_missing_info_file(full_csv_path, output_missing_info_path):
    """
    Đọc file CSV đã hợp nhất, lọc ra các dòng có 'conferenceDates' hoặc 'location' trống
    hoặc chứa các giá trị cụ thể biểu thị thông tin thiếu,
    và lưu các cột 'title', 'acronym' của chúng vào một file mới.

    Args:
        full_csv_path (str): Đường dẫn đến file CSV đầy đủ (đã merge).
        output_missing_info_path (str): Đường dẫn file CSV đầu ra cho dữ liệu thiếu.
    """
    print("\n--- Bắt đầu tạo file chứa các dòng thiếu thông tin (conferenceDates/location) ---")
    try:
        # Đọc file full.csv, thay thế các giá trị NaN bằng chuỗi rỗng để dễ xử lý
        df = pd.read_csv(full_csv_path, dtype=str).fillna('')
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{full_csv_path}' để xử lý.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file '{full_csv_path}': {e}")
        return

    # Kiểm tra xem các cột cần thiết có tồn tại không
    required_columns = ['conferenceDates', 'location', 'title', 'acronym']
    if not all(col in df.columns for col in required_columns):
        print(f"Lỗi: File '{full_csv_path}' thiếu một hoặc nhiều cột cần thiết: {required_columns}.")
        return

    # Định nghĩa các giá trị được coi là thiếu thông tin
    missing_dates_values = ['', 'To be announced', 'TBD', 'To be determined']
    missing_location_values = ['', 'No location', 'TBD']

    # Lọc các dòng mà 'conferenceDates' hoặc 'location' nằm trong danh sách các giá trị thiếu
    missing_info_df = df[(df['conferenceDates'].isin(missing_dates_values)) | 
                         (df['location'].isin(missing_location_values))]

    if missing_info_df.empty:
        print("Không tìm thấy dòng nào có 'conferenceDates' hoặc 'location' trống/thiếu thông tin cụ thể.")
        return

    # Chỉ giữ lại hai cột 'title' và 'acronym'
    final_df = missing_info_df[['title', 'acronym']]

    # Lưu vào file CSV mới
    try:
        final_df.to_csv(output_missing_info_path, index=False, encoding='utf-8-sig')
        print(f"\nĐã tạo thành công file '{output_missing_info_path}' với {len(final_df)} dòng.")
    except Exception as e:
        print(f"Lỗi khi ghi file '{output_missing_info_path}': {e}")

def create_empty_submission_date_file(full_csv_path, output_empty_submission_path, missing_info_csv_path):
    """
    Đọc file CSV đã hợp nhất, lọc ra các dòng có cột 'submissionDate' là "{}",
    và loại trừ những dòng đã có trong file missing_info.csv (dựa trên title và acronym),
    sau đó lưu toàn bộ các cột của chúng vào một file mới.

    Args:
        full_csv_path (str): Đường dẫn đến file CSV đầy đủ (đã merge).
        output_empty_submission_path (str): Đường dẫn file CSV đầu ra cho dữ liệu có 'submissionDate' là "{}".
        missing_info_csv_path (str): Đường dẫn đến file CSV chứa các dòng thiếu thông tin.
    """
    print("\n--- Bắt đầu tạo file chứa các dòng có 'submissionDate' là \"{}\" (trừ những dòng đã trong missing_info) ---")
    try:
        # Đọc file full.csv
        df_full = pd.read_csv(full_csv_path, dtype=str).fillna('')
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{full_csv_path}' để xử lý.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file '{full_csv_path}': {e}")
        return

    # Kiểm tra xem cột 'submissionDate' có tồn tại không
    if 'submissionDate' not in df_full.columns:
        print(f"Lỗi: File '{full_csv_path}' không có cột 'submissionDate'.")
        return
    if 'title' not in df_full.columns or 'acronym' not in df_full.columns:
        print(f"Lỗi: File '{full_csv_path}' thiếu cột 'title' hoặc 'acronym', không thể loại trừ các dòng.")
        return

    # Lọc các dòng mà 'submissionDate' có giá trị chính xác là "{}"
    empty_submission_df = df_full[df_full['submissionDate'] == '{}'].copy() # Tạo bản sao để tránh SettingWithCopyWarning

    if empty_submission_df.empty:
        print("Không tìm thấy dòng nào có 'submissionDate' là \"{}\".")
        return

    # Bước loại trừ: Đọc file missing_info.csv
    try:
        df_missing = pd.read_csv(missing_info_csv_path, dtype=str).fillna('')
        print(f"Đã đọc thành công file missing_info: '{missing_info_csv_path}'.")
    except FileNotFoundError:
        print(f"Cảnh báo: Không tìm thấy file missing_info tại '{missing_info_csv_path}'. Không thực hiện loại trừ.")
        df_missing = pd.DataFrame(columns=['title', 'acronym']) # Tạo DataFrame rỗng để không ảnh hưởng logic tiếp theo
    except Exception as e:
        print(f"Cảnh báo: Lỗi khi đọc file missing_info '{missing_info_csv_path}': {e}. Không thực hiện loại trừ.")
        df_missing = pd.DataFrame(columns=['title', 'acronym'])


    # Tạo một cột kết hợp 'title_acronym' để dễ dàng so sánh
    if 'title' in df_missing.columns and 'acronym' in df_missing.columns:
        empty_submission_df['merge_key'] = empty_submission_df['title'].astype(str) + "_" + empty_submission_df['acronym'].astype(str)
        df_missing['merge_key'] = df_missing['title'].astype(str) + "_" + df_missing['acronym'].astype(str)

        # Loại bỏ các dòng trong empty_submission_df nếu merge_key của chúng tồn tại trong df_missing
        initial_rows = len(empty_submission_df)
        empty_submission_df = empty_submission_df[~empty_submission_df['merge_key'].isin(df_missing['merge_key'])]
        removed_rows = initial_rows - len(empty_submission_df)
        
        if removed_rows > 0:
            print(f"Đã loại bỏ {removed_rows} dòng trùng lặp với missing_info.csv.")
        else:
            print("Không có dòng nào trong empty_submission_date trùng với missing_info để loại bỏ.")

        # Xóa cột merge_key trước khi lưu
        empty_submission_df = empty_submission_df.drop(columns=['merge_key'])
    else:
        print("Cảnh báo: File missing_info.csv không có đủ cột 'title' hoặc 'acronym' để thực hiện loại trừ.")

    if empty_submission_df.empty:
        print("Sau khi loại trừ, không còn dòng nào thỏa mãn điều kiện để tạo file.")
        return

    # Lưu toàn bộ DataFrame đã lọc vào file CSV mới
    try:
        empty_submission_df.to_csv(output_empty_submission_path, index=False, encoding='utf-8-sig')
        print(f"\nĐã tạo thành công file '{output_empty_submission_path}' với {len(empty_submission_df)} dòng.")
    except Exception as e:
        print(f"Lỗi khi ghi file '{output_empty_submission_path}': {e}")

# --- HÀM ĐƯỢC CẬP NHẬT ĐỂ XUẤT CÁC DÒNG KHÔNG ĐƯỢC CẬP NHẬT ---
def update_full_csv_with_recrawled_data(full_csv_path, recrawled_empty_submission_path, output_unmatched_recrawl_path):
    """
    Cập nhật các dòng trong full.csv bằng các dòng tương ứng từ recrawled_empty_submission_path
    dựa trên cặp 'title' và 'acronym'.
    Đồng thời, xuất các dòng từ recrawled_empty_submission_path mà không tìm thấy khớp
    trong full.csv vào một file riêng.

    Args:
        full_csv_path (str): Đường dẫn đến file full.csv cần cập nhật.
        recrawled_empty_submission_path (str): Đường dẫn đến file chứa dữ liệu đã được recrawl
                                                (ví dụ: empty_submission_date.csv sau khi đã được điền dữ liệu).
        output_unmatched_recrawl_path (str): Đường dẫn file CSV đầu ra cho các dòng không khớp.
    """
    print(f"\n--- Bắt đầu cập nhật '{full_csv_path}' bằng dữ liệu từ '{recrawled_empty_submission_path}' ---")
    try:
        df_full = pd.read_csv(full_csv_path, dtype=str).fillna('')
        print(f"Đã đọc file full.csv: {len(df_full)} dòng.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{full_csv_path}'. Không thể cập nhật.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file '{full_csv_path}': {e}. Không thể cập nhật.")
        return

    try:
        df_recrawled = pd.read_csv(recrawled_empty_submission_path, dtype=str).fillna('')
        print(f"Đã đọc file recrawled data: {len(df_recrawled)} dòng.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file '{recrawled_empty_submission_path}'. Không có dữ liệu để cập nhật.")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file '{recrawled_empty_submission_path}': {e}. Không có dữ liệu để cập nhật.")
        return

    # Kiểm tra các cột cần thiết
    required_cols = ['title', 'acronym']
    if not all(col in df_full.columns for col in required_cols):
        print(f"Lỗi: File '{full_csv_path}' thiếu một hoặc nhiều cột cần thiết ({required_cols}).")
        return
    if not all(col in df_recrawled.columns for col in required_cols):
        print(f"Lỗi: File '{recrawled_empty_submission_path}' thiếu một hoặc nhiều cột cần thiết ({required_cols}).")
        return

    # Tạo một cột khóa để merge/cập nhật
    df_full['merge_key'] = df_full['title'].astype(str) + "_" + df_full['acronym'].astype(str)
    df_recrawled['merge_key'] = df_recrawled['title'].astype(str) + "_" + df_recrawled['acronym'].astype(str)

    updated_rows_count = 0
    unmatched_recrawled_rows = []

    # Lặp qua từng dòng trong df_recrawled để cập nhật df_full
    for index_recrawled, row_recrawled in df_recrawled.iterrows():
        merge_key = row_recrawled['merge_key']
        
        # Tìm vị trí của dòng cần cập nhật trong df_full
        matching_rows_indices = df_full[df_full['merge_key'] == merge_key].index
        
        if not matching_rows_indices.empty:
            # Cập nhật tất cả các cột từ row_recrawled vào các dòng tương ứng trong df_full
            cols_to_update = [col for col in df_recrawled.columns if col in df_full.columns and col != 'merge_key']
            
            for col in cols_to_update:
                df_full.loc[matching_rows_indices, col] = row_recrawled[col]
            
            updated_rows_count += len(matching_rows_indices)
        else:
            # Nếu không tìm thấy khớp, thêm dòng này vào danh sách các dòng không khớp
            unmatched_recrawled_rows.append(row_recrawled.drop('merge_key').to_dict())

    # Xóa cột merge_key trước khi lưu
    df_full = df_full.drop(columns=['merge_key'])

    # Lưu DataFrame đã cập nhật vào file full.csv
    try:
        df_full.to_csv(full_csv_path, index=False, encoding='utf-8-sig')
        print(f"\nĐã cập nhật thành công '{updated_rows_count}' dòng trong '{full_csv_path}'.")
        print(f"File '{full_csv_path}' hiện có {len(df_full)} dòng.")
    except Exception as e:
        print(f"Lỗi khi ghi file '{full_csv_path}' sau khi cập nhật: {e}")

    # Xử lý các dòng không khớp
    if unmatched_recrawled_rows:
        df_unmatched = pd.DataFrame(unmatched_recrawled_rows)
        try:
            df_unmatched.to_csv(output_unmatched_recrawl_path, index=False, encoding='utf-8-sig')
            print(f"Đã xuất {len(df_unmatched)} dòng từ '{recrawled_empty_submission_path}' không tìm thấy khớp trong '{full_csv_path}' vào '{output_unmatched_recrawl_path}'.")
        except Exception as e:
            print(f"Lỗi khi ghi file các dòng không khớp '{output_unmatched_recrawl_path}': {e}")
    else:
        print("Không có dòng nào từ file recrawled không tìm thấy khớp trong full.csv.")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # Đặt danh sách các đường dẫn đến file CSV của bạn vào đây.
    input_files = [
        './src/conference/evaluate/tri_tung_all_correct_final.csv',
        './src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv'
    ]

    # Đặt tên cho các file CSV kết quả đầu ra
    output_merged_file = './src/conference/evaluate/full.csv'
    output_missing_file = './src/conference/evaluate/missing_info.csv' # File thiếu conferenceDates/location
    output_empty_submission_file = './src/conference/evaluate/empty_submission_date.csv' # File thiếu submissionDate
    
    # Đường dẫn đến file recrawled data (file empty_submission_date.csv sau khi đã được điền dữ liệu)
    recrawled_empty_submission_file_path = './src/conference/evaluate/evaluate_empty_submission_date.csv' 
    # Đường dẫn cho file chứa các dòng không khớp
    output_unmatched_recrawl_file = './src/conference/evaluate/unmatched_recrawled_entries.csv'

    # --- QUY TRÌNH CHÍNH ---

    # Bước 1: Merge các file CSV và lọc non-link để tạo full.csv ban đầu
    print("Bắt đầu quá trình merge các file CSV và lọc các dòng non-link để tạo full.csv...")
    merge_successful = merge_csv_common_columns_ordered(input_files, output_merged_file)

    if merge_successful:
        # Bước 2: Kiểm tra định dạng JSON trong file full.csv
        validate_json_in_date_columns(output_merged_file)
        
        # Bước 3: Tạo file chứa các dòng thiếu thông tin (conferenceDates/location)
        create_missing_info_file(output_merged_file, output_missing_file)

        # Bước 4: Tạo file chứa các dòng có 'submissionDate' là "{}" và không nằm trong missing_info
        create_empty_submission_date_file(output_merged_file, output_empty_submission_file, output_missing_file)

        # Bước 5: Cập nhật full.csv từ file recrawled empty submission date và xuất các dòng không khớp
        update_full_csv_with_recrawled_data(output_merged_file, recrawled_empty_submission_file_path, output_unmatched_recrawl_file)

    print("\n--- Hoàn thành chương trình ---")

Bắt đầu quá trình merge các file CSV và lọc các dòng non-link để tạo full.csv...
Đã đọc thành công: ./src/conference/evaluate/tri_tung_all_correct_final.csv (có 39 cột)
Đã đọc thành công: ./src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv (có 25 cột)

Tìm thấy 25 cột chung ở tất cả các file:
Thứ tự cột trong file output sẽ dựa trên file cuối cùng: ./src/conference/evaluate/correctFiles/batch_1_6_7_10_11_17.csv
Thứ tự các cột: ['requestId', 'originalRequestId', 'title', 'acronym', 'mainLink', 'cfpLink', 'impLink', 'information', 'conferenceDates', 'year', 'location', 'cityStateProvince', 'country', 'continent', 'type', 'submissionDate', 'notificationDate', 'cameraReadyDate', 'registrationDate', 'otherDate', 'topics', 'publisher', 'summary', 'callForPapers', 'Unnamed: 24']
DataFrame đã hợp nhất ban đầu có 887 dòng.

Đã hợp nhất và lọc thành công các file vào './src/conference/evaluate/full.csv'.
File kết quả có 887 dòng và 25 cột.

--- Bắt đầu kiểm tra định dạng JSON trong c

In [52]:
import pandas as pd
import re

def normalize_text(text):
    """
    Chuẩn hóa văn bản bằng cách loại bỏ nội dung bên trong cặp dấu ngoặc đơn
    và chính cặp dấu ngoặc đơn, đồng thời đảm bảo chỉ có một khoảng trắng
    giữa các từ.
    """
    if pd.isna(text):
        return ""

    cleaned_text = re.sub(r'\s*\(.*?\)\s*', ' ', str(text))
    final_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return final_text

def find_unique_in_core_detailed(full_csv_path, core_csv_path, non_link_acronyms_path, skip_acronyms_path):
    """
    Tìm các dòng chỉ có trong file CORE_2023.csv dựa trên cặp (title, acronym)
    sau khi chuẩn hóa, và in ra các bước chi tiết.
    Đồng thời, lọc bỏ các conference trong full.csv có trong non_link.txt và skip.txt.

    Args:
        full_csv_path (str): Đường dẫn đến file full.csv.
        core_csv_path (str): Đường dẫn đến file CORE_2023.csv.
        non_link_acronyms_path (str): Đường dẫn đến file TXT chứa các key non-link.
        skip_acronyms_path (str): Đường dẫn đến file TXT chứa các key cần bỏ qua (skip).

    Returns:
        pandas.DataFrame: DataFrame chứa các dòng chỉ có trong CORE_2023.csv
                          sau khi đã lọc bỏ các conference trong non_link và skip.
    """

    print("--- BƯỚC 1: ĐỌC DỮ LIỆU ---")
    print(f"Đang đọc file full.csv từ: {full_csv_path}")
    try:
        df_full = pd.read_csv(full_csv_path)
        if 'title' not in df_full.columns or 'acronym' not in df_full.columns:
            raise ValueError("File full.csv phải có các cột 'title' và 'acronym'.")
        print("Đã đọc full.csv thành công. 5 dòng đầu tiên:")
        print(df_full.head().to_string())
        print(f"Tổng số dòng trong full.csv: {len(df_full)}")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file full.csv tại đường dẫn: {full_csv_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Lỗi khi đọc full.csv: {e}")
        return pd.DataFrame()

    print(f"\nĐang đọc file CORE_2023.csv từ: {core_csv_path}")
    try:
        df_core = pd.read_csv(core_csv_path, header=None)
        if df_core.shape[1] < 3:
            raise ValueError("File CORE_2023.csv phải có ít nhất 3 cột để lấy title và acronym.")
        df_core.rename(columns={1: 'title', 2: 'acronym'}, inplace=True)
        print("Đã đọc CORE_2023.csv thành công. 5 dòng đầu tiên (cột 1 là title, cột 2 là acronym):")
        print(df_core.head().to_string())
        print(f"Tổng số dòng trong CORE_2023.csv: {len(df_core)}")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CORE_2023.csv tại đường dẫn: {core_csv_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Lỗi khi đọc CORE_2023.csv: {e}")
        return pd.DataFrame()

    print("\n--- BƯỚC 2: CHUẨN HÓA DỮ LIỆU ---")
    print("Áp dụng hàm normalize_text cho cột 'title' và 'acronym' của cả hai DataFrame.")

    df_full['normalized_title'] = df_full['title'].apply(normalize_text)
    df_full['normalized_acronym'] = df_full['acronym'].apply(normalize_text)
    print("\n5 dòng đầu tiên của full.csv sau khi chuẩn hóa:")
    print(df_full[['title', 'normalized_title', 'acronym', 'normalized_acronym']].head().to_string())

    df_core['normalized_title'] = df_core['title'].apply(normalize_text)
    df_core['normalized_acronym'] = df_core['acronym'].apply(normalize_text)
    print("\n5 dòng đầu tiên của CORE_2023.csv sau khi chuẩn hóa:")
    print(df_core[['title', 'normalized_title', 'acronym', 'normalized_acronym']].head().to_string())

    print("\n--- BƯỚC 3: TẠO KHÓA SO SÁNH (KEY) ---")
    print("Tạo cột 'key' từ cặp (normalized_title, normalized_acronym) cho cả hai DataFrame.")
    df_full['key'] = list(zip(df_full['normalized_title'], df_full['normalized_acronym']))
    df_core['key'] = list(zip(df_core['normalized_title'], df_core['normalized_acronym']))

    print("\n5 dòng đầu tiên của full.csv với cột 'key':")
    print(df_full[['title', 'acronym', 'key']].head().to_string())
    print("\n5 dòng đầu tiên của CORE_2023.csv với cột 'key':")
    print(df_core[['title', 'acronym', 'key']].head().to_string())

    print("\n--- BƯỚC 4: ĐỌC VÀ XỬ LÝ DANH SÁCH NON-LINK VÀ SKIP ---")
    non_link_keys = set()
    try:
        with open(non_link_acronyms_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(',')
                if len(parts) >= 3:
                    title = normalize_text(parts[1])
                    acronym = normalize_text(parts[2])
                    if title and acronym:
                        non_link_keys.add((title, acronym))
        print(f"Đã đọc {len(non_link_keys)} key non-link từ {non_link_acronyms_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file non-link acronyms tại đường dẫn: {non_link_acronyms_path}. Bỏ qua bước lọc này.")
    except Exception as e:
        print(f"Lỗi khi đọc file non-link acronyms: {e}. Bỏ qua bước lọc này.")

    skip_keys = set()
    try:
        with open(skip_acronyms_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(',')
                if len(parts) >= 3:
                    title = normalize_text(parts[1])
                    acronym = normalize_text(parts[2])
                    if title and acronym:
                        skip_keys.add((title, acronym))
        print(f"Đã đọc {len(skip_keys)} key skip từ {skip_acronyms_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file skip acronyms tại đường dẫn: {skip_acronyms_path}. Bỏ qua bước lọc này.")
    except Exception as e:
        print(f"Lỗi khi đọc file skip acronyms: {e}. Bỏ qua bước lọc này.")

    print("\n--- BƯỚC 5: LỌC BỎ CÁC CONFERENCE TRONG full.csv CÓ TRONG NON-LINK VÀ SKIP ---")
    initial_full_count = len(df_full)
    print(f"Số dòng ban đầu trong full.csv: {initial_full_count}")

    # Lọc bỏ các conference có key trong danh sách non-link
    df_full_filtered = df_full[~df_full['key'].isin(non_link_keys)].copy()
    print(f"Số dòng trong full.csv sau khi loại bỏ non-link: {len(df_full_filtered)} (Đã loại bỏ {initial_full_count - len(df_full_filtered)} dòng).")

    # Lọc bỏ các conference có key trong danh sách skip
    df_full_filtered = df_full_filtered[~df_full_filtered['key'].isin(skip_keys)].copy()
    print(f"Số dòng trong full.csv sau khi loại bỏ skip: {len(df_full_filtered)} (Đã loại bỏ {initial_full_count - len(df_full_filtered)} dòng tổng cộng).")

    print("\n--- BƯỚC 6: TẠO TẬP HỢP CÁC KEY TỪ full.csv ĐÃ LỌC ---")
    keys_in_full_filtered = set(df_full_filtered['key'])
    print(f"Tổng số key duy nhất trong full.csv (sau khi lọc non-link và skip): {len(keys_in_full_filtered)}")
    # In ra một vài key mẫu để kiểm tra
    print("Một vài key mẫu từ full.csv (đã chuẩn hóa và lọc):")
    for i, key in enumerate(list(keys_in_full_filtered)[:5]):
        print(f"  - {key}")

    print("\n--- BƯỚC 7: LỌC CÁC DÒNG CHỈ CÓ TRONG CORE_2023.csv ---")
    print("Lọc df_core để tìm các dòng mà 'key' của chúng KHÔNG có trong tập hợp key của full.csv đã lọc.")
    unique_in_core_df = df_core[~df_core['key'].isin(keys_in_full_filtered)].copy()

    print(f"\nTìm thấy {len(unique_in_core_df)} dòng chỉ có trong CORE_2023.csv.")
    if not unique_in_core_df.empty:
        print("5 dòng đầu tiên của kết quả (các dòng chỉ có trong CORE_2023.csv):")
        print(unique_in_core_df.head().to_string())
    else:
        print("Không tìm thấy dòng nào chỉ có trong CORE_2023.csv.")

    print("\n--- BƯỚC 8: DỌN DẸP CÁC CỘT TẠM THỜI ---")
    # Xóa các cột tạm thời đã tạo
    unique_in_core_df.drop(columns=['normalized_title', 'normalized_acronym', 'key'], inplace=True, errors='ignore')

    print("Đã xóa các cột 'normalized_title', 'normalized_acronym', 'key' khỏi DataFrame kết quả.")
    print("Quá trình so sánh hoàn tất.")

    return unique_in_core_df

if __name__ == "__main__":
    # --- Cấu hình đường dẫn file của bạn ---
    FULL_CSV_FILE = './src/conference/evaluate/full.csv'
    CORE_CSV_FILE = './src/conference/csv/CORE_2023.csv'
    NON_LINK_ACRONYMS_FILE = './non_link.txt'
    SKIP_ACRONYMS_FILE = './skip.txt'
    OUTPUT_CSV_FILE = './src/conference/evaluate/recrawl_all_core.csv'

    # Gọi hàm để tìm các dòng duy nhất với chi tiết
    result_df = find_unique_in_core_detailed(FULL_CSV_FILE, CORE_CSV_FILE, NON_LINK_ACRONYMS_FILE, SKIP_ACRONYMS_FILE)

    if not result_df.empty:
        print(f"\n--- KẾT QUẢ CUỐI CÙNG ---")
        print(f"Tổng số dòng duy nhất trong CORE_2023.csv: {len(result_df)}")
        print(f"Kết quả đã được lưu vào file: {OUTPUT_CSV_FILE}")
        result_df.to_csv(OUTPUT_CSV_FILE, index=False)
    else:
        print("\n--- KẾT QUẢ CUỐI CÙNG ---")
        print("Không tìm thấy dòng nào chỉ có trong CORE_2023.csv.")

--- BƯỚC 1: ĐỌC DỮ LIỆU ---
Đang đọc file full.csv từ: ./src/conference/evaluate/full.csv
Đã đọc full.csv thành công. 5 dòng đầu tiên:
                      requestId originalRequestId                                                                        title acronym                                      mainLink                                                                                 cfpLink                                                       impLink                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [53]:
import pandas as pd
import re

# Sử dụng lại hàm normalize_text đã được cải tiến
def normalize_text(text):
    """
    Chuẩn hóa văn bản bằng cách loại bỏ nội dung bên trong cặp dấu ngoặc đơn
    và chính cặp dấu ngoặc đơn, đồng thời đảm bảo chỉ có một khoảng trắng
    giữa các từ.
    """
    if pd.isna(text):
        return ""

    cleaned_text = re.sub(r'\s*\(.*?\)\s*', ' ', str(text))
    final_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return final_text

def filter_recrawl_list_with_non_links(recrawl_all_core_path, crawling_files_paths, non_link_acronyms_path, skip_acronyms_path, output_csv_path):
    """
    Lọc danh sách recrawl_all_core.csv bằng cách loại bỏ các conference
    đang được crawl (dựa trên 3 file CSV khác), các conference
    có acronym trong danh sách non-link từ file TXT, và các conference
    có acronym trong danh sách skip từ file TXT.

    Args:
        recrawl_all_core_path (str): Đường dẫn đến file recrawl_all_core.csv.
        crawling_files_paths (list): Danh sách các đường dẫn đến các file CSV
                                      chứa danh sách conference đang được crawl.
        non_link_acronyms_path (str): Đường dẫn đến file TXT chứa các acronym non-link.
        skip_acronyms_path (str): Đường dẫn đến file TXT chứa các acronym cần bỏ qua (skip).
        output_csv_path (str): Đường dẫn để lưu file CSV kết quả cuối cùng.
    """
    print("--- BƯỚC 1: ĐỌC DANH SÁCH CẦN RECRAWL TỪ recrawl_all_core.csv ---")
    try:
        # Đọc recrawl_all_core.csv - giả định nó đã có cột 'title' và 'acronym'
        df_recrawl = pd.read_csv(recrawl_all_core_path)
        if 'title' not in df_recrawl.columns or 'acronym' not in df_recrawl.columns:
            raise ValueError(f"File {recrawl_all_core_path} phải có các cột 'title' và 'acronym'.")
        print(f"Đã đọc {len(df_recrawl)} dòng từ {recrawl_all_core_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file: {recrawl_all_core_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc {recrawl_all_core_path}: {e}")
        return

    print("\n--- BƯỚC 2: CHUẨN HÓA VÀ TẠO KEY CHO DANH SÁCH RECRAWL ---")
    df_recrawl['normalized_title'] = df_recrawl['title'].apply(normalize_text)
    df_recrawl['normalized_acronym'] = df_recrawl['acronym'].apply(normalize_text)
    df_recrawl['key'] = list(zip(df_recrawl['normalized_title'], df_recrawl['normalized_acronym']))
    recrawl_keys = set(df_recrawl['key'])
    print(f"Tổng số key duy nhất trong recrawl_all_core: {len(recrawl_keys)}")

    print("\n--- BƯỚC 3: ĐỌC VÀ TỔNG HỢP CÁC CONFERENCE ĐANG ĐƯỢC CRAWL ---")
    all_crawling_keys = set()
    for i, file_path in enumerate(crawling_files_paths):
        print(f"Đang đọc file crawling {i+1}/{len(crawling_files_paths)}: {file_path}")
        try:
            # Đối với các file crawling, nếu chúng cũng là CSV với cột 'title' và 'acronym'
            df_crawling = pd.read_csv(file_path)
            if 'title' not in df_crawling.columns or 'acronym' not in df_crawling.columns:
                print(f"Cảnh báo: File {file_path} không có cột 'title' hoặc 'acronym'. Bỏ qua file này.")
                continue

            df_crawling['normalized_title'] = df_crawling['title'].apply(normalize_text)
            df_crawling['normalized_acronym'] = df_crawling['acronym'].apply(normalize_text)
            df_crawling['key'] = list(zip(df_crawling['normalized_title'], df_crawling['normalized_acronym']))

            all_crawling_keys.update(set(df_crawling['key']))
            print(f"  Đã thêm {len(set(df_crawling['key']))} key từ {file_path}. Tổng số key đang crawl: {len(all_crawling_keys)}")

        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file crawling tại đường dẫn: {file_path}. Bỏ qua file này.")
        except Exception as e:
            print(f"Lỗi khi đọc file crawling {file_path}: {e}. Bỏ qua file này.")

    print(f"\nTổng số key duy nhất từ tất cả các file đang crawl: {len(all_crawling_keys)}")

    print("\n--- BƯỚC 4: ĐỌC DANH SÁCH ACRONYM NON-LINK VÀ SKIP (Đã điều chỉnh cho định dạng TXT mới) ---")
    non_link_keys = set() # Thay đổi từ set acronyms thành set keys
    try:
        with open(non_link_acronyms_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(',') # Tách dòng bằng dấu phẩy
                if len(parts) >= 3: # Đảm bảo có đủ các cột
                    title = normalize_text(parts[1]) # Cột 2 là title
                    acronym = normalize_text(parts[2]) # Cột 3 là acronym
                    if title and acronym: # Đảm bảo không thêm key trống
                        non_link_keys.add((title, acronym))
        print(f"Đã đọc {len(non_link_keys)} key non-link từ {non_link_acronyms_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file non-link acronyms tại đường dẫn: {non_link_acronyms_path}. Bỏ qua bước lọc này.")
    except Exception as e:
        print(f"Lỗi khi đọc file non-link acronyms: {e}. Bỏ qua bước lọc này.")

    skip_keys = set() # Thay đổi từ set acronyms thành set keys
    try:
        with open(skip_acronyms_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(',') # Tách dòng bằng dấu phẩy
                if len(parts) >= 3: # Đảm bảo có đủ các cột
                    title = normalize_text(parts[1]) # Cột 2 là title
                    acronym = normalize_text(parts[2]) # Cột 3 là acronym
                    if title and acronym: # Đảm bảo không thêm key trống
                        skip_keys.add((title, acronym))
        print(f"Đã đọc {len(skip_keys)} key skip từ {skip_acronyms_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file skip acronyms tại đường dẫn: {skip_acronyms_path}. Bỏ qua bước lọc này.")
    except Exception as e:
        print(f"Lỗi khi đọc file skip acronyms: {e}. Bỏ qua bước lọc này.")

    print("\n--- BƯỚC 5: LỌC DANH SÁCH RECRAWL CUỐI CÙNG ---")
    print("Tìm các conference trong recrawl_all_core mà KHÔNG có trong danh sách đang crawl,")
    print("KHÔNG có key trong danh sách non-link, VÀ KHÔNG có key trong danh sách skip.")

    # Bắt đầu với danh sách recrawl_all_core
    current_recrawl_df = df_recrawl.copy()
    initial_count = len(current_recrawl_df)
    print(f"Số dòng ban đầu trong danh sách recrawl: {initial_count}")

    # Lọc bỏ các conference đang được crawl
    filtered_by_crawling = current_recrawl_df[~current_recrawl_df['key'].isin(all_crawling_keys)].copy()
    print(f"Số dòng sau khi loại bỏ các conference đang crawl: {len(filtered_by_crawling)} (Đã loại bỏ {initial_count - len(filtered_by_crawling)} dòng).")

    # Lọc bỏ các conference có key trong danh sách non-link
    # Bây giờ chúng ta lọc dựa trên 'key' thay vì 'normalized_acronym'
    filtered_by_non_link = filtered_by_crawling[~filtered_by_crawling['key'].isin(non_link_keys)].copy()
    print(f"Số dòng sau khi loại bỏ các conference non-link: {len(filtered_by_non_link)} (Đã loại bỏ {len(filtered_by_crawling) - len(filtered_by_non_link)} dòng).")

    # Lọc bỏ các conference có key trong danh sách skip
    # Tương tự, lọc dựa trên 'key'
    final_recrawl_list_df = filtered_by_non_link[~filtered_by_non_link['key'].isin(skip_keys)].copy()
    print(f"Số dòng sau khi loại bỏ các conference skip: {len(final_recrawl_list_df)} (Đã loại bỏ {len(filtered_by_non_link) - len(final_recrawl_list_df)} dòng).")

    print(f"\nTìm thấy {len(final_recrawl_list_df)} conference cần recrawl cuối cùng.")

    print("\n--- BƯỚC 6: DỌN DẸP VÀ LƯU KẾT QUẢ ---")
    # Xóa các cột tạm thời đã tạo
    final_recrawl_list_df.drop(columns=['normalized_title', 'normalized_acronym', 'key'], inplace=True, errors='ignore')

    if not final_recrawl_list_df.empty:
        print(f"5 dòng đầu tiên của danh sách recrawl cuối cùng:")
        print(final_recrawl_list_df.head().to_string())
        print(f"\nKết quả đã được lưu vào file: {output_csv_path}")
        final_recrawl_list_df.to_csv(output_csv_path, index=False)
    else:
        print("Không tìm thấy conference nào cần recrawl cuối cùng sau khi lọc.")

if __name__ == "__main__":
    # --- Cấu hình đường dẫn file của bạn ---
    RECRAWL_ALL_CORE_FILE = './src/conference/evaluate/recrawl_all_core.csv'

    # Danh sách các file CSV chứa conference đang được crawl
    # Đảm bảo các file này có cột 'title' và 'acronym'
    CRAWLING_FILES = [
    #    './src/conference/evaluate/tri_tung_all_recrawl_final.csv'
    ]

    # Đường dẫn đến file TXT chứa các acronym non-link
    # File này bây giờ được mong đợi có định dạng: id,title,acronym,...
    NON_LINK_ACRONYMS_FILE = './non_link.txt'

    # Đường dẫn đến file TXT chứa các acronym cần bỏ qua (skip)
    # File này cũng được mong đợi có định dạng: id,title,acronym,...
    SKIP_ACRONYMS_FILE = './skip.txt'

    OUTPUT_FINAL_RECRAWL_FILE = './src/conference/evaluate/final_recrawl_list.csv'

    # Gọi hàm để lọc danh sách recrawl
    filter_recrawl_list_with_non_links(RECRAWL_ALL_CORE_FILE, CRAWLING_FILES, NON_LINK_ACRONYMS_FILE, SKIP_ACRONYMS_FILE, OUTPUT_FINAL_RECRAWL_FILE)

--- BƯỚC 1: ĐỌC DANH SÁCH CẦN RECRAWL TỪ recrawl_all_core.csv ---
Đã đọc 95 dòng từ ./src/conference/evaluate/recrawl_all_core.csv.

--- BƯỚC 2: CHUẨN HÓA VÀ TẠO KEY CHO DANH SÁCH RECRAWL ---
Tổng số key duy nhất trong recrawl_all_core: 95

--- BƯỚC 3: ĐỌC VÀ TỔNG HỢP CÁC CONFERENCE ĐANG ĐƯỢC CRAWL ---

Tổng số key duy nhất từ tất cả các file đang crawl: 0

--- BƯỚC 4: ĐỌC DANH SÁCH ACRONYM NON-LINK VÀ SKIP (Đã điều chỉnh cho định dạng TXT mới) ---
Đã đọc 69 key non-link từ ./non_link.txt.
Đã đọc 9 key skip từ ./skip.txt.

--- BƯỚC 5: LỌC DANH SÁCH RECRAWL CUỐI CÙNG ---
Tìm các conference trong recrawl_all_core mà KHÔNG có trong danh sách đang crawl,
KHÔNG có key trong danh sách non-link, VÀ KHÔNG có key trong danh sách skip.
Số dòng ban đầu trong danh sách recrawl: 95
Số dòng sau khi loại bỏ các conference đang crawl: 95 (Đã loại bỏ 0 dòng).
Số dòng sau khi loại bỏ các conference non-link: 32 (Đã loại bỏ 63 dòng).
Số dòng sau khi loại bỏ các conference skip: 23 (Đã loại bỏ 9 dòng).

T

In [54]:
import pandas as pd

def merge_and_select_columns(full_csv_path, unique_csv_path, output_csv_path):
    """
    Gộp hai file CSV (full.csv và unique_in_CORE_2023.csv) và chỉ giữ lại
    các cột 'title' và 'acronym', sau đó lưu vào một file CSV mới.

    Args:
        full_csv_path (str): Đường dẫn đến file full.csv.
        unique_csv_path (str): Đường dẫn đến file unique_in_CORE_2023.csv.
        output_csv_path (str): Đường dẫn để lưu file CSV kết quả.
    """
    print(f"Đang đọc file full.csv từ: {full_csv_path}")
    try:
        df_full = pd.read_csv(full_csv_path)
        # Đảm bảo các cột 'title' và 'acronym' tồn tại
        if 'title' not in df_full.columns or 'acronym' not in df_full.columns:
            raise ValueError(f"File {full_csv_path} phải có các cột 'title' và 'acronym'.")
        print("Đã đọc full.csv thành công.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file full.csv tại đường dẫn: {full_csv_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc full.csv: {e}")
        return

    print(f"Đang đọc file unique_in_CORE_2023.csv từ: {unique_csv_path}")
    try:
        df_unique = pd.read_csv(unique_csv_path)
        # Đảm bảo các cột 'title' và 'acronym' tồn tại
        if 'title' not in df_unique.columns or 'acronym' not in df_unique.columns:
            raise ValueError(f"File {unique_csv_path} phải có các cột 'title' và 'acronym'.")
        print("Đã đọc unique_in_CORE_2023.csv thành công.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file unique_in_CORE_2023.csv tại đường dẫn: {unique_csv_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc unique_in_CORE_2023.csv: {e}")
        return

    print("Đang chọn các cột 'title' và 'acronym' từ cả hai DataFrame...")
    # Chọn chỉ các cột 'title' và 'acronym' từ mỗi DataFrame
    df_full_selected = df_full[['title', 'acronym']]
    df_unique_selected = df_unique[['title', 'acronym']]

    print("Đang gộp hai DataFrame...")
    # Gộp hai DataFrame theo chiều dọc (thêm hàng)
    # ignore_index=True để reset index của DataFrame kết quả
    merged_df = pd.concat([df_full_selected, df_unique_selected], ignore_index=True)

    print(f"Tổng số dòng sau khi gộp: {len(merged_df)}")

    # Tùy chọn: Xóa các dòng trùng lặp nếu bạn muốn một danh sách duy nhất
    # Dòng trùng lặp ở đây có nghĩa là cả title và acronym đều giống hệt nhau
    # Nếu bạn muốn giữ lại tất cả các dòng, kể cả trùng lặp, hãy bỏ qua bước này
    print("Đang loại bỏ các dòng trùng lặp (nếu có)...")
    initial_rows = len(merged_df)
    merged_df.drop_duplicates(inplace=True)
    rows_after_dedup = len(merged_df)
    print(f"Số dòng sau khi loại bỏ trùng lặp: {rows_after_dedup} (Đã loại bỏ {initial_rows - rows_after_dedup} dòng trùng lặp).")


    print(f"Đang lưu kết quả vào file: {output_csv_path}")
    try:
        merged_df.to_csv(output_csv_path, index=False)
        print("Đã lưu file thành công.")
    except Exception as e:
        print(f"Lỗi khi lưu file: {e}")

if __name__ == "__main__":
    FULL_CSV_FILE = './src/conference/evaluate/full.csv'
    UNIQUE_CSV_FILE = './src/conference/evaluate/recrawl_all_core.csv'
    OUTPUT_MERGED_FILE = './src/conference/evaluate/merged_full_and_unique.csv'

    # Gọi hàm để gộp các file
    merge_and_select_columns(FULL_CSV_FILE, UNIQUE_CSV_FILE, OUTPUT_MERGED_FILE)

Đang đọc file full.csv từ: ./src/conference/evaluate/full.csv
Đã đọc full.csv thành công.
Đang đọc file unique_in_CORE_2023.csv từ: ./src/conference/evaluate/recrawl_all_core.csv
Đã đọc unique_in_CORE_2023.csv thành công.
Đang chọn các cột 'title' và 'acronym' từ cả hai DataFrame...
Đang gộp hai DataFrame...
Tổng số dòng sau khi gộp: 959
Đang loại bỏ các dòng trùng lặp (nếu có)...
Số dòng sau khi loại bỏ trùng lặp: 955 (Đã loại bỏ 4 dòng trùng lặp).
Đang lưu kết quả vào file: ./src/conference/evaluate/merged_full_and_unique.csv
Đã lưu file thành công.


In [51]:
import pandas as pd

def find_and_export_duplicates(input_csv_path, output_csv_path, subset_columns=None):
    """
    Tìm và xuất các dòng trùng lặp trong một file CSV.

    Args:
        input_csv_path (str): Đường dẫn đến file CSV đầu vào (ví dụ: full.csv).
        output_csv_path (str): Đường dẫn để lưu file CSV chứa các dòng trùng lặp.
        subset_columns (list, optional): Danh sách các tên cột để kiểm tra trùng lặp.
                                         Nếu None, tất cả các cột sẽ được sử dụng.
                                         Ví dụ: ['title', 'acronym']
    """
    print(f"Đang đọc file CSV từ: {input_csv_path}")
    try:
        df = pd.read_csv(input_csv_path)
        print(f"Đã đọc {len(df)} dòng từ {input_csv_path}.")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file CSV tại đường dẫn: {input_csv_path}")
        return
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return

    print("Đang tìm kiếm các dòng trùng lặp...")

    # Tìm các dòng trùng lặp
    # keep=False: Đánh dấu TẤT CẢ các lần xuất hiện của một dòng trùng lặp là True
    #             (bao gồm cả lần xuất hiện đầu tiên).
    #             Nếu muốn chỉ đánh dấu các bản sao (không bao gồm bản gốc đầu tiên),
    #             sử dụng keep='first' hoặc keep='last'.
    # subset: Các cột để kiểm tra trùng lặp. Nếu None, kiểm tra tất cả các cột.
    duplicate_rows = df[df.duplicated(subset=subset_columns, keep=False)]

    if not duplicate_rows.empty:
        print(f"Tìm thấy {len(duplicate_rows)} dòng trùng lặp.")
        print(f"Đang lưu các dòng trùng lặp vào file: {output_csv_path}")
        try:
            duplicate_rows.to_csv(output_csv_path, index=False)
            print("Đã lưu file trùng lặp thành công.")
        except Exception as e:
            print(f"Lỗi khi lưu file trùng lặp: {e}")
    else:
        print("Không tìm thấy dòng trùng lặp nào.")

if __name__ == "__main__":
    # --- Cấu hình đường dẫn file của bạn ---
    INPUT_CSV_FILE = './src/conference/evaluate/full.csv'
    OUTPUT_DUPLICATES_FILE = 'full_duplicates.csv'

    # --- Cấu hình các cột để kiểm tra trùng lặp ---
    # Nếu bạn muốn kiểm tra trùng lặp dựa trên TẤT CẢ các cột, hãy để là None
    # COLUMNS_TO_CHECK = None

    # Nếu bạn muốn kiểm tra trùng lặp chỉ dựa trên 'title' và 'acronym', hãy sử dụng:
    COLUMNS_TO_CHECK = ['title', 'acronym']

    # Gọi hàm để tìm và xuất các dòng trùng lặp
    find_and_export_duplicates(INPUT_CSV_FILE, OUTPUT_DUPLICATES_FILE, COLUMNS_TO_CHECK)

Đang đọc file CSV từ: ./src/conference/evaluate/full.csv
Đã đọc 864 dòng từ ./src/conference/evaluate/full.csv.
Đang tìm kiếm các dòng trùng lặp...
Không tìm thấy dòng trùng lặp nào.
